<a href="https://colab.research.google.com/github/JoaoPauloSarzedasRibeiro/data_manipulation_with_Python/blob/main/Spark_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalando o PySpark no Google Colab

In [ ]:
# instalar as dependências necessárias para o Spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

#Setup

In [ ]:
# iniciar uma sessão local chamada spark
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from os.path import abspath

warehouse_location = abspath('../data/spark-warehouse')

spark = (
    SparkSession
    .builder
    .master('local[*]')
    .config("spark.driver.memory", '8g')
    .config("spark.sql.warehouse.dir", warehouse_location)
    .enableHiveSupport()
    .getOrCreate()
)

#Importando dados

In [ ]:
#Adicionando o arquivo CSV hospedado no GitHub para que o Spark consiga acessa-lo
url = 'https://raw.githubusercontent.com/peetck/IMDB-Top1000-Movies/master/IMDB-Movie-Data.csv'
from pyspark import SparkFiles
spark.sparkContext.addFile(url)

In [ ]:
#Lendo os dados em CSV
df_csv = (
    spark
    .read
    .format('csv')
    .options(header=True, inferSchema=True,sep=',',encoding='latin1')
    .load(SparkFiles.get('IMDB-Movie-Data.csv'))
    )

In [ ]:
df_csv.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Actors: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Runtime (Minutes): string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Votes: string (nullable = true)
 |-- Revenue (Millions): double (nullable = true)
 |-- Metascore: double (nullable = true)



In [ ]:
df_csv = (
    df_csv
    .withColumnRenamed('Runtime (Minutes)','Runtime_Minutes')
    .withColumnRenamed('Revenue (Millions)', 'Revenue_Millions')
)

In [ ]:
#Salvando os dados no formato parquet para manipulação com Spark
df_csv.write.format('parquet').mode('overwrite').save('df_titles')

In [ ]:
#Criando um novo DataFrame com os dados já no formato Parquet
df_titles = spark.read.format('parquet').load('df_titles')

In [ ]:
df_titles.limit(5).toPandas()

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime_Minutes,Rating,Votes,Revenue_Millions,Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0


#Databases e Catalog

O catálogo de metadados do Spark pode ser acessado pelo objeto

`SparkSession.catalog`

As principais funcionalidades são:

* `listDatabases()`: lista todas os databases disponíveis;
* `listTables()`: lista todas as tabelas disponíveis em um determinado database;
* `listFucntions()`: lista as funções disponíveis em um determinado database;
* `refreshTable()`: atualiza os metadados de uma determinada tabela
* `uncacheTable()`: remove uma tabela salva em memória
* `clearCache()`: remove todas as tabelas salvas em memória

Os databases do Spark são uma ferramenta para organizar tabelas. Eles podem e devem ser vistos como algo muito próximo dos databases de servidores de bancos de dados relacionais. O Spark utiliza por padrão um database chamado default, que serve para criar tabelas, views e realizar consultas caso o usuário não tenha definido o seu próprio. Um ponto importante é que essas estruturas persistem em diferentes sessões: se o usuário mudar de database, todas as tabelas permanecerão no database anterior e vão precisar ser consultadas de maneira diferente.

Existem alguns comandos do SQL importantes na hora de se trabalhar com databases. Else são:

* `SHOW DATABASES`: lista todas os databases disponíveis, de forma análoga ao Catalog ;
* `CREATE DATABASE <nome_do_db>`: cria um database
* `USE <nome_do_db>`: define o database como o atual para a realização de queries
  * **Obs**: ao se mudar de database, é possível acessar tabelas de um database anterior usando o prefixo “nome_do_db.” antes do nome da tabela.
Exemplo:

        USE db2
        SELECT * FROM db1.table
        
* `SELECT current_database()`: retorna qual o database definido como o atual
*` DROP DATABASE IF EXISTS <nome_do_db>`: deleta determinado database dentre aqueles que foram definidos. Atenção: nunca delete o database default do Spark.

#Tabelas e Views

###Tabelas

* **Managed Tables**: o Spark administra tanto os dados quanto os metadados das tabelas, de forma que operações como DROP TABLE afetam também os dados escritos em disco;
* **Unmanaged Tables**: o Spark administra somente os metadados da tabela, e os dados escritos em disco não são alterados em nenhum momento.

In [ ]:
df_titles.limit(5).toPandas()

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime_Minutes,Rating,Votes,Revenue_Millions,Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0


In [ ]:
df_titles_sample = df_titles.sample(fraction = 0.5)

**Criando Managed Tables**

In [ ]:
# Usando DataFrame
df_titles_sample.write.saveAsTable("title_basics_managed")

Usando SQL: 
`CREATE TABLE nome_tabela (schema)`

**Criando Unmanaged Tables**

In [ ]:
# Usando DataFrame
df_titles_sample.write.option('path', '../data/imdb/title_basics_unmanaged').saveAsTable("title_basics_unmanaged")

Usando SQL: `CREATE EXTERNAL TABLE nome_tabela (schema) USING parquet OPTIONS (path 'caminho_arquivo')`



In [ ]:
spark.catalog.listTables()

[Table(name='title_basics_managed', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='title_basics_unmanaged', database='default', description=None, tableType='EXTERNAL', isTemporary=False)]

###Views

**Criando Views**

In [ ]:
df_titles_sample.createOrReplaceTempView('title_basics_view')

Usando SQL: `CREATE OR REPLACE TEMP VIEW AS title_basics_view SELECT * FROM <nome da tabela>`

**Criando Views Globais**

In [ ]:
df_titles_sample.createOrReplaceGlobalTempView('title_basics_global_view')

Usando SQL: `CREATE OR REPLACE GLOBAL TEMP VIEW AS title_basics_global_view SELECT * FROM <nome da tabela>`

In [ ]:
# Podemos verificar na lista de tabelas que as views ficam destacadas como isTemporary=True
spark.catalog.listTables()

[Table(name='title_basics_managed', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='title_basics_unmanaged', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='title_basics_view', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

**Deletando Views**

In [ ]:
spark.catalog.dropTempView("title_basics_view")

In [ ]:
spark.catalog.dropGlobalTempView("title_basics_global_view")

##Acessando a interface de Queries

In [ ]:
spark.sql('SHOW DATABASES').toPandas()

,databaseName
0,default


In [ ]:
spark.sql('SHOW TABLES').toPandas()

,database,tableName,isTemporary
0,default,title_basics_managed,False
1,default,title_basics_unmanaged,False


In [ ]:
spark.sql('CREATE DATABASE db_teste')

DataFrame[]

In [ ]:
spark.sql('USE db_teste')

DataFrame[]

In [ ]:
spark.sql("""
CREATE TABLE title_basics_managed 
(Rank INT,
 Title STRING, 
 Genre STRING, 
 Description STRING,
 Director STRING,
 Actors STRING,
 Year STRING, 
 Runtime_Minutes STRING,
 Rating STRING,
 Votes INT,
 Revenue_Millions DOUBLE,
 Metascore DOUBLE)
""").toPandas()

""


In [ ]:
spark.sql('SHOW TABLES').toPandas()

,database,tableName,isTemporary
0,db_teste,title_basics_managed,False


In [ ]:
spark.sql("""
            INSERT INTO title_basics_managed  SELECT * FROM default.title_basics_managed
          """
).toPandas()

""


In [ ]:
spark.sql('SELECT * FROM title_basics_managed limit 5').toPandas()

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime_Minutes,Rating,Votes,Revenue_Millions,Metascore
0,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7,485820,126.46,65.0
1,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0
2,7,La La Land,"Comedy,Drama,Music",A jazz pianist falls for an aspiring actress i...,Damien Chazelle,"Ryan Gosling, Emma Stone, Rosemarie DeWitt, J....",2016,128,8.3,258682,151.06,93.0
3,10,Passengers,"Adventure,Drama,Romance",A spacecraft traveling to a distant colony pla...,Morten Tyldum,"Jennifer Lawrence, Chris Pratt, Michael Sheen,...",2016,116,7,192177,100.01,41.0
4,11,Fantastic Beasts and Where to Find Them,"Adventure,Family,Fantasy",The adventures of writer Newt Scamander in New...,David Yates,"Eddie Redmayne, Katherine Waterston, Alison Su...",2016,133,7.5,232072,234.02,66.0


In [ ]:
spark.sql('DROP TABLE title_basics_managed').toPandas()

""


In [ ]:
spark.sql('USE DEFAULT').toPandas()

""


In [ ]:
spark.sql('DROP TABLE title_basics_unmanaged').toPandas()

""


In [ ]:
spark.sql('SHOW TABLES').toPandas()

,database,tableName,isTemporary
0,default,title_basics_managed,False


In [ ]:
# Exemplo de como podemos misturar as APIs da SQL e pySpark/DataFrames sem nenhum problema
spark.sql('SELECT CAST(Runtime_Minutes as INT) FROM title_basics_view')\
.withColumn('teste', f.col('Runtime_Minutes') + 1).limit(5).toPandas()

,Runtime_Minutes,teste
0,124,125
1,123,124
2,128,129
3,116,117
4,133,134
